[구글 코랩(Colab)에서 실행하기](https://colab.research.google.com/github/lovedlim/tensorflow/blob/main/Part%203/3.5_callback.ipynb)

In [1]:
'''
import tensorflow as tf

# 케라스의 내장 데이터셋에서 mnist 데이터셋을 로드
mnist = tf.keras.datasets.mnist

# load_data()로 데이터셋을 로드 합니다.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 로드된 데이터셋 확인
print('train set: ', x_train.shape, y_train.shape)
print('test  set: ', x_test.shape, y_test.shape)

# 데이터 정규화
x_train = x_train / x_train.max() # max: 255
x_test = x_test / x_test.max() # max: 255
'''

"\nimport tensorflow as tf\n\n# 케라스의 내장 데이터셋에서 mnist 데이터셋을 로드\nmnist = tf.keras.datasets.mnist\n\n# load_data()로 데이터셋을 로드 합니다.\n(x_train, y_train), (x_test, y_test) = mnist.load_data()\n\n# 로드된 데이터셋 확인\nprint('train set: ', x_train.shape, y_train.shape)\nprint('test  set: ', x_test.shape, y_test.shape)\n\n# 데이터 정규화\nx_train = x_train / x_train.max() # max: 255\nx_test = x_test / x_test.max() # max: 255\n"

In [2]:
import numpy as np
import tensorflow as tf

# 파일 경로 설정 (환경에 맞게 수정하세요)
mnist_path = '../dataset/mnist/mnist.npz'  # 윈도우 예시
#mnist_path = 'D:/datasets/mnist.npz'  # 윈도우 예시
# mnist_path = '/home/user/datasets/mnist.npz'  # 리눅스 예시

# npz 파일 직접 로드
with np.load(mnist_path) as data:
    x_train = data['x_train']
    y_train = data['y_train']
    x_test = data['x_test']
    y_test = data['y_test']

# 데이터 확인
print('train set: ', x_train.shape, y_train.shape)
print('test  set: ', x_test.shape, y_test.shape)

# 정규화
x_train = x_train / x_train.max()
x_test = x_test / x_test.max()

print("x_train[0, 10:15, 10:15] = " + repr(x_train[0, 10:15, 10:15]))
print("x_test[0, 10:15, 10:15] = " + repr(x_test[0, 10:15, 10:15]))

train set:  (60000, 28, 28) (60000,)
test  set:  (10000, 28, 28) (10000,)
x_train[0, 10:15, 10:15] = array([[0.00392157, 0.60392157, 0.99215686, 0.35294118, 0.        ],
       [0.        , 0.54509804, 0.99215686, 0.74509804, 0.00784314],
       [0.        , 0.04313725, 0.74509804, 0.99215686, 0.2745098 ],
       [0.        , 0.        , 0.1372549 , 0.94509804, 0.88235294],
       [0.        , 0.        , 0.        , 0.31764706, 0.94117647]])
x_test[0, 10:15, 10:15] = array([[0.        , 0.06666667, 0.25882353, 0.05490196, 0.2627451 ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ]])


## 5-1. 모델 체크포인트

In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), 
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(32, activation='relu'), 
    tf.keras.layers.Dense(10, activation='softmax'), # 노드 10개로 생성
])

In [4]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
# 체크포인트 설정
checkpoint = tf.keras.callbacks.ModelCheckpoint(#filepath='tmp_checkpoint.ckpt', 
                                                filepath='tmp_checkpoint.weights.h5',  # 확장자 수정
                                                save_weights_only=True, 
                                                save_best_only=True, 
                                                monitor='val_loss', 
                                                verbose=1)


In [6]:
model.fit(x_train, y_train, 
          validation_data=(x_test, y_test), 
          epochs=10, 
          callbacks=[checkpoint]
          )

Epoch 1/10
1864/1875 [============================>.] - ETA: 0s - loss: 0.2246 - accuracy: 0.9320
Epoch 1: val_loss improved from inf to 0.11391, saving model to tmp_checkpoint.weights.h5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2241 - accuracy: 0.9322 - val_loss: 0.1139 - val_accuracy: 0.9643
Epoch 2/10
1852/1875 [============================>.] - ETA: 0s - loss: 0.0954 - accuracy: 0.9705
Epoch 2: val_loss improved from 0.11391 to 0.08453, saving model to tmp_checkpoint.weights.h5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0957 - accuracy: 0.9704 - val_loss: 0.0845 - val_accuracy: 0.9742
Epoch 3/10
1868/1875 [============================>.] - ETA: 0s - loss: 0.0669 - accuracy: 0.9790
Epoch 3: val_loss improved from 0.08453 to 0.08190, saving model to tmp_checkpoint.weights.h5
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0670 - accuracy: 0.9790 - val_loss: 0.0819 - val_accuracy: 0.9752
Epoch 4/10
1862/1875 [=======

In [7]:
# 모델 체크포인트 로드 전
loss, acc = model.evaluate(x_test, y_test)
print(f'체크포인트 로드 전: loss: {loss:.4f}, acc: {acc:.4f}')

# 체크포인트 파일을 모델에 로드
#model.load_weights('tmp_checkpoint.ckpt')
model.load_weights('tmp_checkpoint.weights.h5')
loss, acc = model.evaluate(x_test, y_test)
print(f'체크포인트 로드 후: loss: {loss:.4f}, acc: {acc:.4f}')

313/313 [==============================] - 0s 1ms/step - loss: 0.1061 - accuracy: 0.9790
체크포인트 로드 전: loss: 0.1061, acc: 0.9790
313/313 [==============================] - 0s 981us/step - loss: 0.0784 - accuracy: 0.9790
체크포인트 로드 후: loss: 0.0784, acc: 0.9790


## 5-2. 조기종료

In [8]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), 
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(32, activation='relu'), 
    tf.keras.layers.Dense(10, activation='softmax'), # 노드 10개로 생성
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
# EarlyStopping 콜백 생성
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

In [10]:
model.fit(x_train, y_train, 
          validation_data=(x_test, y_test), 
          epochs=20, 
          callbacks=[earlystopping]
          )

Epoch 1/20
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2347 - accuracy: 0.9305 - val_loss: 0.1104 - val_accuracy: 0.9647
Epoch 2/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0937 - accuracy: 0.9717 - val_loss: 0.0829 - val_accuracy: 0.9750
Epoch 3/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0646 - accuracy: 0.9796 - val_loss: 0.0838 - val_accuracy: 0.9750
Epoch 4/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0499 - accuracy: 0.9841 - val_loss: 0.0850 - val_accuracy: 0.9743
Epoch 5/20
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0422 - accuracy: 0.9866 - val_loss: 0.0912 - val_accuracy: 0.9747


## 5-3. 학습률 스케줄러

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), 
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(32, activation='relu'), 
    tf.keras.layers.Dense(10, activation='softmax'), # 노드 10개로 생성
])

In [12]:
'''
def scheduler(epoch, lr):
    tf.print(f'learning_rate: {lr:.5f}')
    # 첫 5 에포크 동안 유지
    if epoch < 5:
        return lr
    else:
    # 학습률 감소 적용
        return lr * tf.math.exp(-0.1)
'''

def scheduler(epoch, lr):
    print(f'learning_rate: {lr:.5f}')  # tf.print → print 로 변경
    if epoch < 5:
        return float(lr)  # 명시적으로 float 반환
    else:
        return float(lr * tf.math.exp(-0.1))  # float으로 변환

# 콜백 객체생성 및 scheduler 함수 적용
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler)

In [13]:
model.compile(tf.keras.optimizers.SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# 초기 학습률 확인(0.01)
#print(round(model.optimizer.lr.numpy(), 5))
print(round(model.optimizer.learning_rate.numpy(), 5))

0.01


In [14]:
model.fit(x_train, y_train, 
          validation_data=(x_test, y_test), 
          epochs=10,
          # 학습률 스케줄러 적용
          callbacks=[lr_scheduler]
          )
# 최종 학습률 스케줄러 확인
#round(model.optimizer.lr.numpy(), 5)
round(model.optimizer.learning_rate.numpy(), 5)

learning_rate: 0.01000
Epoch 1/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.6269 - accuracy: 0.8313 - val_loss: 0.2798 - val_accuracy: 0.9198 - lr: 0.0100
learning_rate: 0.01000
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2499 - accuracy: 0.9286 - val_loss: 0.2082 - val_accuracy: 0.9398 - lr: 0.0100
learning_rate: 0.01000
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1927 - accuracy: 0.9448 - val_loss: 0.1693 - val_accuracy: 0.9502 - lr: 0.0100
learning_rate: 0.01000
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1575 - accuracy: 0.9548 - val_loss: 0.1462 - val_accuracy: 0.9560 - lr: 0.0100
learning_rate: 0.01000
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.1326 - accuracy: 0.9617 - val_loss: 0.1322 - val_accuracy: 0.9599 - lr: 0.0100
learning_rate: 0.01000
Epoch 6/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0

0.00607

## 5-4. 텐서보드

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)), 
    tf.keras.layers.Dense(256, activation='relu'), 
    tf.keras.layers.Dense(64, activation='relu'), 
    tf.keras.layers.Dense(32, activation='relu'), 
    tf.keras.layers.Dense(10, activation='softmax'), # 노드 10개로 생성
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [16]:
# 텐서보드 저장 경로 지정
log_dir = 'tensorboard'

# 텐서보드 콜백 정의
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x_train, y_train,
            validation_data=(x_test, y_test), 
            epochs=10, 
            callbacks=[tensorboard],
            )

Epoch 1/10
1875/1875 [==============================] - 7s 3ms/step - loss: 0.2290 - accuracy: 0.9311 - val_loss: 0.1194 - val_accuracy: 0.9619
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0948 - accuracy: 0.9711 - val_loss: 0.0833 - val_accuracy: 0.9743
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0641 - accuracy: 0.9799 - val_loss: 0.0749 - val_accuracy: 0.9767
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0508 - accuracy: 0.9845 - val_loss: 0.0727 - val_accuracy: 0.9781
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0405 - accuracy: 0.9869 - val_loss: 0.0893 - val_accuracy: 0.9753
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0315 - accuracy: 0.9898 - val_loss: 0.0800 - val_accuracy: 0.9788
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0285 - accuracy: 0.9909 - val_loss: 0.0816 - val_accuracy:

In [1]:
# 텐서보드 extension 로드
%load_ext tensorboard

In [2]:
# 텐서보드 출력 매직 커멘드
%tensorboard --logdir {log_dir}
#%tensorboard --logdir=log_dir
#%tensorboard --logdir=your_log_dir
